In [1]:
import os
PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
os.chdir(PROJ_ROOT)

In [2]:
import numpy as np
from numpy.linalg import inv as inv

In [3]:
import configparser
from configparser import ExtendedInterpolation

In [4]:
def kr_prod(a, b):
    return np.einsum('ir, jr -> ijr', a, b).reshape(a.shape[0] * b.shape[0], -1)

def cp_combine(U, V, X):
    return np.einsum('is, js, ts -> ijt', U, V, X)

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

In [64]:
# ALS algorithm for CP completion 
def CP_ALS(sparse_tensor_input, rank, maxiter, test_info = None):
    
    sparse_tensor = sparse_tensor_input.copy()
    dim1, dim2, dim3 = sparse_tensor.shape
    dim = np.array([dim1, dim2, dim3])
    U = 0.1 * np.random.rand(dim1, rank)
    V = 0.1 * np.random.rand(dim2, rank)
    X = 0.1 * np.random.rand(dim3, rank)
    
    pos = np.where(sparse_tensor != 0)
    binary_tensor = np.zeros((dim1, dim2, dim3))
    binary_tensor[pos] = 1
    tensor_hat = np.zeros((dim1, dim2, dim3))
    
    min_test_cls = 999
    min_test_cls_iteration = -1
    
    for iters in range(maxiter):
        for order in range(dim.shape[0]):
            if order == 0:
                var1 = kr_prod(X, V).T
            elif order == 1:
                var1 = kr_prod(X, U).T
            else:
                var1 = kr_prod(V, U).T
            var2 = kr_prod(var1, var1)
            var3 = np.matmul(var2, ten2mat(binary_tensor, order).T).reshape([rank, rank, dim[order]])
            var4 = np.matmul(var1, ten2mat(sparse_tensor, order).T)
            for i in range(dim[order]):
                var_Lambda = var3[ :, :, i]
                inv_var_Lambda = inv((var_Lambda + var_Lambda.T)/2 + 10e-12 * np.eye(rank))
                vec = np.matmul(inv_var_Lambda, var4[:, i])
                if order == 0:
                    U[i, :] = vec.copy()
                elif order == 1:
                    V[i, :] = vec.copy()
                else:
                    X[i, :] = vec.copy()

        tensor_hat = cp_combine(U, V, X)

        if (iters + 1) % 10 == 0:
            mape = np.sum(np.abs(sparse_tensor[pos] - tensor_hat[pos])/np.abs(sparse_tensor[pos]))/sparse_tensor[pos].shape[0]
            mape = np.sum(np.abs(sparse_tensor[pos] - tensor_hat[pos])/np.abs(sparse_tensor[pos]))/sparse_tensor[pos].shape[0]
            rmse = np.sqrt(np.sum((sparse_tensor[pos] - tensor_hat[pos]) ** 2)/sparse_tensor[pos].shape[0])
            print('Iter: {}'.format(iters + 1))
            print('Training MAPE: {:.6}'.format(mape))
            print('Training RMSE: {:.6}'.format(rmse))
            print()
            
            if test_info is not None:
                # print test mape and rmse 
                test_pos_tuple,  test_values = test_info
                
                norm_tcs = np.linalg.norm(test_values)
                error_tcs = np.linalg.norm(tensor_hat[test_pos_tuple] - test_values)
                test_tcs = error_tcs/norm_tcs
                
                test_rmse = np.sqrt(np.sum((test_values - tensor_hat[test_pos_tuple]) ** 2)\
                                    /test_values.shape[0])
                print('Testing TCS: {:.6}'.format(test_tcs))
                print('Testing RMSE: {:.6}'.format(test_rmse))
                print()
                
                ## stop iteration if smallest test_tcs get 
                
                if test_tcs < min_test_cls:
                    min_test_cls = test_tcs
                    min_test_cls_iteration = iters
                
                else:
                    if ((iters - min_test_cls_iteration) > 30):
                        break 
                    
    return tensor_hat, U, V, X, min_test_cls, min_test_cls_iteration

In [65]:
def choose_test_index_for_location(location_array, test_ratio = 0.2):
    np.random.seed(2020)
    location_choice = np.random.choice(location_array, size=int(len(location_array)*0.2), replace=False)
    return location_choice

In [66]:
def choose_test_index(pos, num_of_locations=50):
    test_index = np.array([])
    for loc_num in range(num_of_locations):
        location_array = np.where(pos[0] == loc_num)[0]
        location_choice = choose_test_index_for_location(location_array)
        location_choice.sort()
        test_index =  np.concatenate((test_index, location_choice), axis=0)
    return test_index

In [67]:
config_path = 'config/parameters.ini'
# read configuration from config file
pars = configparser.ConfigParser(interpolation=ExtendedInterpolation())
pars.read(config_path)

['config/parameters.ini']

In [68]:
import pickle 
location_search_tensor_path = 'data/external/location_search_tensor.pkl'
location_tensor_array = pickle.load(open(location_search_tensor_path, 'rb'))
sparse_tensor = np.nan_to_num(location_tensor_array, copy=True, nan=0)
# create test sparse_tensor 
pos = np.where(sparse_tensor != 0)

In [69]:
test_index = choose_test_index(pos)
test_index = list(test_index.astype(int))
test_pos_tuple = tuple([pos[0][test_index], pos[1][test_index], pos[2][test_index]])
test_values = sparse_tensor[test_pos_tuple] 
sparse_tensor[test_pos_tuple] = 0

In [77]:
print(len(np.where(sparse_tensor != 0)[0]))
print(len(test_pos_tuple[0]))

75104
18753


In [78]:
import time
rank = 10

In [79]:
rank_info_dict = {}

In [80]:
rank_info_dict

{}

In [81]:
for rank in [5, 10, 20, 40, 80]:
    start = time.time()
    print('Rank: ' + str(rank))
    print()
    maxiter = 700
    np.random.seed(10)
    tensor_hat, U, V, X, min_test_cls, min_test_cls_iteration = CP_ALS(sparse_tensor, rank, maxiter, test_info = (test_pos_tuple, test_values))
    end = time.time()
    print('Testing TCS: {:.6}'.format(min_test_cls))
    print('Testing iter: {}'.format(min_test_cls_iteration))
    print('Running time: %d seconds'%(end - start))
    rank_info_dict[rank] = [min_test_cls, min_test_cls_iteration]
    
    

Rank: 5

Iter: 10
Training MAPE: 0.468262
Training RMSE: 4.82425

Testing TCS: 0.189894
Testing RMSE: 5.16104

Iter: 20
Training MAPE: 0.463717
Training RMSE: 4.75637

Testing TCS: 0.186969
Testing RMSE: 5.08151

Iter: 30
Training MAPE: 0.463306
Training RMSE: 4.70886

Testing TCS: 0.184614
Testing RMSE: 5.01751

Iter: 40
Training MAPE: 0.464035
Training RMSE: 4.66374

Testing TCS: 0.182592
Testing RMSE: 4.96255

Iter: 50
Training MAPE: 0.463631
Training RMSE: 4.6301

Testing TCS: 0.181555
Testing RMSE: 4.93437

Iter: 60
Training MAPE: 0.462611
Training RMSE: 4.61161

Testing TCS: 0.181199
Testing RMSE: 4.92471

Iter: 70
Training MAPE: 0.46243
Training RMSE: 4.60265

Testing TCS: 0.181075
Testing RMSE: 4.92134

Iter: 80
Training MAPE: 0.462326
Training RMSE: 4.59811

Testing TCS: 0.181031
Testing RMSE: 4.92013

Iter: 90
Training MAPE: 0.462237
Training RMSE: 4.59563

Testing TCS: 0.181004
Testing RMSE: 4.91941

Iter: 100
Training MAPE: 0.462159
Training RMSE: 4.59419

Testing TCS: 0.18

Iter: 30
Training MAPE: 0.316785
Training RMSE: 2.77758

Testing TCS: 0.193692
Testing RMSE: 5.26425

Iter: 40
Training MAPE: 0.314616
Training RMSE: 2.7563

Testing TCS: 0.204789
Testing RMSE: 5.56584

Iter: 50
Training MAPE: 0.313358
Training RMSE: 2.74297

Testing TCS: 0.209278
Testing RMSE: 5.68785

Testing TCS: 0.164964
Testing iter: 9
Running time: 7 seconds
Rank: 40

Iter: 10
Training MAPE: 0.282768
Training RMSE: 2.11266

Testing TCS: 0.222368
Testing RMSE: 6.04361

Iter: 20
Training MAPE: 0.27703
Training RMSE: 2.03768

Testing TCS: 0.241116
Testing RMSE: 6.55316

Iter: 30
Training MAPE: 0.274048
Training RMSE: 2.00156

Testing TCS: 0.25812
Testing RMSE: 7.01531

Iter: 40
Training MAPE: 0.272316
Training RMSE: 1.97674

Testing TCS: 0.269066
Testing RMSE: 7.31281

Iter: 50
Training MAPE: 0.271255
Training RMSE: 1.9584

Testing TCS: 0.275031
Testing RMSE: 7.47491

Testing TCS: 0.222368
Testing iter: 9
Running time: 20 seconds
Rank: 80

Iter: 10
Training MAPE: 0.239285
Training R

In [87]:
import pandas as pd
rank_info_pd = pd.DataFrame(rank_info_dict).T


In [92]:
min_row = rank_info_pd[rank_info_pd[0] == rank_info_pd[0].min()]

In [97]:
best_rank = min_row.index[0]

In [100]:
best_iter = int(min_row[1] + 1) 

In [103]:
best_rank

10

In [104]:
loc_tensor = np.nan_to_num(location_tensor_array, copy=True, nan=0)
# create test sparse_tensor 
pos = np.where(loc_tensor != 0)
print(len(pos[0]))


93857


In [105]:
np.random.seed(10)
tensor_hat, U, V, X, min_test_cls, min_test_cls_iteration = CP_ALS(loc_tensor, best_rank, best_iter, test_info = None)


Iter: 10
Training MAPE: 0.369839
Training RMSE: 3.67186

Iter: 20
Training MAPE: 0.362667
Training RMSE: 3.62693

Iter: 30
Training MAPE: 0.360433
Training RMSE: 3.61304



In [110]:
imputed_save_path = 'data/external/imputed_tensor_array.pkl'

In [111]:
pickle.dump(tensor_hat, open(imputed_save_path, 'wb'))

In [48]:
np.random.rand(10, 10)

array([[0.28208046, 0.42902209, 0.60881578, 0.10529745, 0.71388675,
        0.92436679, 0.3215174 , 0.08982099, 0.61858301, 0.27757761],
       [0.44894571, 0.29109628, 0.38945601, 0.24276079, 0.31831308,
        0.80394262, 0.08823857, 0.06748663, 0.39247303, 0.4635031 ],
       [0.18785696, 0.50746823, 0.66365047, 0.36725844, 0.64431549,
        0.9936906 , 0.5693492 , 0.54378324, 0.62786684, 0.94811119],
       [0.77188159, 0.47028695, 0.01429415, 0.27996584, 0.41644882,
        0.90234388, 0.81468965, 0.42867465, 0.48564143, 0.74732602],
       [0.28409757, 0.09296883, 0.09294327, 0.52782895, 0.6138759 ,
        0.53584455, 0.68153912, 0.89757469, 0.97227988, 0.26726937],
       [0.26003433, 0.95209319, 0.78470666, 0.06388887, 0.80737222,
        0.67839269, 0.94759288, 0.66188065, 0.10587892, 0.62708692],
       [0.31896491, 0.62993584, 0.45765399, 0.32925796, 0.9206549 ,
        0.75684575, 0.77050882, 0.93704153, 0.3552869 , 0.15349886],
       [0.22491033, 0.70717823, 0.4190647

In [ ]:
rank_info_dict

In [ ]:
min_test_cls

In [ ]:
min_test_cls_iteration

In [ ]:
def tenerror(fitx, realx, omega):
    """
    Calculate Three Kinds of Error
    :param fitx: fitted tensor
    :param realx: ground-truth tensor
    :param omega: index tensor of observed entries
    """

#     if type(omega) != np.ndarray and type(omega) != pyten.tenclass.Tensor:
#         raise ValueError("AirCP: cannot recognize the format of observed Tensor!")
#     elif type(omega) == pyten.tenclass.Tensor:
#         omega = omega.tondarray
#     if type(realx) == np.ndarray:
#         realx = pyten.tenclass.Tensor(realx)
#     if type(fitx) == np.ndarray:
#         realx = pyten.tenclass.Tensor(realx)
    norm1 = np.linalg.norm(realx)
    norm2 = np.linalg.norm(realx * (1 - omega))
    err1 = np.linalg.norm(fitx - realx)  # Absolute Error
    err2 = np.linalg.norm((fitx - realx) * (1 - omega))
    re_err1 = err1 / norm1  # Relative Error 1
    re_err2 = err2 / norm2  # Relative Error 2
    return err1, re_err1, re_err2